In [1]:
import torch
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pysr
import math

/home/elg227/.conda/envs/bnn_chaos_model/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labels = ['time', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno', 'a1', 'e1', 'i1', 'cos_Omega1', 'sin_Omega1', 'cos_pomega1', 'sin_pomega1', 'cos_theta1', 'sin_theta1', 'a2', 'e2', 'i2', 'cos_Omega2', 'sin_Omega2', 'cos_pomega2', 'sin_pomega2', 'cos_theta2', 'sin_theta2', 'a3', 'e3', 'i3', 'cos_Omega3', 'sin_Omega3', 'cos_pomega3', 'sin_pomega3', 'cos_theta3', 'sin_theta3', 'm1', 'm2', 'm3', 'nan_mmr_near', 'nan_mmr_far', 'nan_megno']

# not all of these labels are actually used. for training, these inputs are zeroed out, but still passed in as zeroes.
# ideally, the linear layer ignores them, which does happen if i do l1 regularization to it
skipped = ['nan_mmr_near', 'nan_mmr_far', 'nan_megno', 'e+_near', 'e-_near', 'max_strength_mmr_near', 'e+_far', 'e-_far', 'max_strength_mmr_far', 'megno']

In [8]:
import pickle

# Path to the pickle file
pickle_file_path = 'sr_results/37967.pkl'

# Load the pickle file
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Print the contents of the pickle file
# print(data)
# print(type(data.equations_))
# print([type(s) for s in data.equations_])
print([s for s in data.equations_])
print(data.equations_['equation'])

['complexity', 'loss', 'score', 'equation', 'sympy_format', 'lambda_format']
0                                             6.7952027
1                                       (7.40236 - s11)
2                     ((s14 ^ -0.34515792) - -3.673654)
3             (((s14 ^ -0.3431603) - -4.3165917) - s11)
4        (((s14 ^ -0.34238997) - -4.020728) - sin(m19))
5     (((s14 ^ -0.4097145) - -4.020728) - (1.5008097...
6     ((((s14 ^ -0.36284095) - -4.020728) - (1.50080...
7     ((((s14 ^ -0.4097145) - -4.020728) - (1.500809...
8     (((s14 ^ -0.36284095) - -4.020728) - ((1.50080...
9     ((((s14 ^ -0.3431603) - -4.020728) - -0.927278...
10    ((((s14 ^ -0.3431603) - -4.020728) - -0.927278...
11    ((((s14 ^ -0.3431603) - -4.020728) - (sin(m5 *...
12    ((((s14 ^ -0.3431603) - -4.020728) - (sin(m5 *...
13    (((((s14 ^ -0.3431603) - -4.020728) - sin(m5 *...
14    (((((s14 ^ -0.3431603) - -4.020728) - sin(m5 *...
15    (((((s14 ^ -0.34238997) - -4.020728) - (sin(m5...
16    (((((s14 ^ -0.3431603

In [3]:
# l1 reg = 2: 95944
# feature_nn = torch.load('models/95944_feature_nn.pt')

# l1 reg = 0.2: 92122
# feature_nn = torch.load('models/92122_feature_nn.pt')

# l1 nonabs reg: 63524
# feature_nn = torch.load('models/63524_feature_nn.pt')

# this gives the (n_features, n_inputs) matrix of the linear transformation used as f1
# input_linear = feature_nn.weight.detach().numpy()
# input_bias = feature_nn.bias.detach().numpy()
# input_linear.shape

In [4]:
# topk 2 pruned masked linear
# feature_nn = torch.load('models/52410_feature_nn.pt')

# topk2 pruned masked linear 3750
# feature_nn = torch.load('models/57964_feature_nn.pt')

# topk2 pruned masked linear 7500 again
# feature_nn = torch.load('models/2762_feature_nn.pt')

# another one
# feature_nn = torch.load('models/55549_feature_nn.pt')

# products3 cos only
# feature_nn = torch.load('models/7923_feature_nn.pt')

# products3 sin&cos
feature_nn = torch.load('models/8218_feature_nn.pt')
feature_nn = feature_nn[1]

TypeError: 'Linear' object is not subscriptable

In [6]:
from modules import Products3
model = Products3()
import torch.nn as nn 
f1 = nn.Sequential(Products3(), feature_nn)
feature_nn = f1
print(feature_nn)
feature_nn = f1[1]
print(type(feature_nn))

Sequential(
  (0): Products3()
  (1): Linear(in_features=41, out_features=20, bias=False)
)
<class 'torch.nn.modules.linear.Linear'>


In [12]:
#input_linear.shape
input_linear = feature_nn.linear.weight * feature_nn.mask
l = input_linear.abs().sum(dim=-1)
l[l < 1] = 0
print(l[l > 1])
_, ixs = torch.topk(input_linear.abs().sum(dim=-1), 10)
l[~ixs] = 0
print(l)


AttributeError: 'Linear' object has no attribute 'linear'

In [5]:
input_linear = feature_nn.linear.weight * feature_nn.mask
input_linear = input_linear.detach().numpy()
if feature_nn.linear.bias is not None:
    input_bias = feature_nn.linear.bias.detach().numpy()
else:
    input_bias = np.zeros(input_linear.shape[0])

AttributeError: 'Sequential' object has no attribute 'linear'

In [7]:
# m_i is the mean of the i'th feature, s_i is the standard deviation
# get the linear transformation that creates feature i
def linear_transformation(i):
    return input_linear[i]

In [8]:

# let's make the linear transformation a bit easier to read
def format_num(x):
    if abs(x) > 0.1:
        return f'{x:.2f}'
    if abs(x) > 0.01:
        return f'{x:.3f}'
    elif abs(x) > 0.001:
        return f'{x:.4f}'
    else:
        return f'{x:.2e}'

format_vec = np.vectorize(format_num)

In [9]:
# now we can write it as a combination of the input features
# we'll sort the features by their absolute value to make it a bit easier to read
def feature_equation(i):
    transformation = linear_transformation(i)
    bias = input_bias[i]
    sorted_ixs = np.argsort(np.abs(transformation))[::-1]
    features = [format_num(transformation[i]) + ' * ' + labels[i] for i in sorted_ixs if transformation[i] != 0]
    if bias != 0:
        features = [format_num(bias)] + features

    return features

In [10]:
for i in range(input_linear.shape[0]):
    print(str(i) + ": " + ' + '.join(feature_equation(i)))

NameError: name 'input_linear' is not defined

In [11]:
def features_used(i):
    transformation = linear_transformation(i)
    sorted_ixs = np.argsort(np.abs(transformation))[::-1]
    return ', '.join([labels[i] for i in sorted_ixs if transformation[i] != 0])

def feature_equation2(i):
    transformation = linear_transformation(i)
    bias = input_bias[i]
    sorted_ixs = np.argsort(np.abs(transformation))[::-1]
    v = [format_num(bias)]
    for i in sorted_ixs:
        if transformation[i] != 0:
            v.append(format_num(transformation[i]))
            v.append(labels[i])

    return v

In [12]:
ixs = [(i, features_used(i)) for i in range(input_linear.shape[0])]
sorted_ixs = sorted(ixs, key=lambda x: x[1])
values = []
for i, _ in sorted_ixs:
    print(str(i) + ", " + ' , '.join(feature_equation(i)))
    values.append([i] + feature_equation2(i))

NameError: name 'input_linear' is not defined

In [ ]:
# values3 = values
# values2 = values
# values1 = values

In [ ]:
feature_dict = {}
all_values = values3 + values2 + values1
for value_list, i in (values3, 3), (values2, 2), (values1, 1):
    for l in value_list:
        f = l[-3] + ', ' + l[-1]
        if f not in feature_dict:
            feature_dict[f] = [(1, 0), (2, 0), (3, 0)]
        feature_dict[f][i-1] = (i, [l[1], l[2], l[4]])


In [ ]:
for f in feature_dict:
    print(f)
    for i in range(3):
        print('\t' + str(feature_dict[f][i]))





In [ ]:
s = [(f, sum([0 if not l[1] else sum([abs(float(f1)) for f1 in l[1]]) for l in feature_dict[f]])) for f in feature_dict]

In [ ]:
s = sorted(s, key=lambda x: -x[1])

In [ ]:
for t in s:
    f = t[0]
    print(f)
    for i in range(3):
        print('\t' + str(feature_dict[f][i]))
